In [4]:
import numpy as np
from matplotlib import pyplot as plt

folder_path = 'data/'               # Путь в папку где лежат исходные данные
detector_in_radius = 0.015          # Внутренний радиус детекторов
detector_out_radius = 0.025         # Внешний радиус детекторов
detector_distances = [1, 1.7, 2.5]  # Расстояния установки детекторов
time_accuracy = 0.05                # Точность с которой пишется время прилета в нс
n_angles = 8                        # На сколько сегментов детектор разбивается по углам
n_radii = 4                         # На сколько сегментов детектор разбивается по
augmentation = True                 # Добаввлять аугментацию? Да - True; Нет - False
maximum_hits = 80                   # Максимальное количество частиц, попавших в детекторы в одном событии. Для разбиения 8х4 следует ставить 80-100
n_lines = 3                         # Количество линий информации. Первая - время прилета, Вторая - номер ячейки детектора, Третья - номер детектора
name_of_file = 'PD070_test'         # Имя файла, в который будет записана информация. Записывается в папку data/ с расширением .dat

In [5]:
# mass in GeV/c^2
proton_mass = 0.938272
masses_dict = {'211': 0.139570, '2212': 0.938272, '3112': 1.197449, '321': 0.493677, '3222': 1.18937,
              '3312': 1.32171, '3334': 1.67245, '-211': 0.139570, '-2212': 0.938272, '-3112': 1.197449, 
              '-321': 0.493677, '-3222': 1.18937, '-3312': 1.32171, '-3334': 1.67245}
# Определение массы частицы по ее индексу
def get_particle_mass(p_index: str) -> float:
    if len(p_index) != 10:
        return masses_dict[p_index]
    else:
        m = (int(p_index) % 10000) // 10
        return float(m * proton_mass)
        
# Вычисление скорости частицы по ее импульсу и индексу   
def calc_velocity(p: float, pz: float, ptype: str) -> float:
    return abs(pz) / np.sqrt(get_particle_mass(ptype) **2 + p**2 / 1**2)

# Вычисление времени пролета по скорости и расстоянию до детектора
def calc_time(v: float, distance: float) -> float:
    c = 2.99722458 * 10**8
    velocity = v * c
    time = distance / velocity
    return time

# Поворот на угол, задаваемый случайным числом randomizer
def random_rotation(angle, max_angle, randomizer):
    new_angle = angle + randomizer * max_angle
    if (new_angle == np.pi) or (new_angle == -np.pi):
        new_angle -= 0.00000001
    if new_angle > np.pi:
        new_angle -= 2*np.pi
    elif new_angle < -np.pi:
        new_angle += 2*np.pi
    return new_angle
   

In [6]:
n_rings = len(detector_distances)
events = []

# Чтение 11 файлов с данными из папки 
for i in range(1, 12):        # iterate over all 11 given files of simulated events (from 1 to 11)
    filename = folder_path + "data_" + str(i) + ".dat"
    with open(filename,'r') as inpf: 
        l = inpf.readline()
        l = inpf.readline()
        l = inpf.readline()
        l = inpf.readline()
        l = inpf.readline()
        
        for l in inpf.readlines():
            s=l.strip().split()
            if len(s) == 5:
                n,npart,b,bx,by = int(s[0]),int(s[1]),float(s[2]),float(s[3]),float(s[4])
                # 
                hits = []
                for j in range(n_rings):
                    hits.append([[],[]])     # for every detector, negative and positive Pz side
                phi_b = np.arctan2(bx,by)
                events.append([b,phi_b,hits])
            elif len(s) == 11:
                p_charge = int(s[0].strip())
                ptype = s[4].strip()
                # if len(ptype) > 5:
                    # ptype = '1000'
                px = float(s[5])
                py = float(s[6])
                pz = float(s[7])
                p_along = abs(pz)
                p_across = np.sqrt(px**2+py**2)
                p_full = np.sqrt(p_across **2 + p_along **2)
                
                # Обработка всех частиц с ненулевым зарядом
                if p_charge != 0:
                    for ind in range(n_rings):    # for every detector check if particle hit it
                        under_upper_bound = (p_across / p_along) < (detector_out_radius / detector_distances[ind])
                        above_lower_bound = (p_across / p_along) > (detector_in_radius / detector_distances[ind])
                        hit_detector =  under_upper_bound and above_lower_bound
                        if hit_detector:
                            pnorm=np.sqrt(px**2+py**2+pz**2)
                            pxn=px/pnorm
                            pyn=py/pnorm
                            p_velocity = calc_velocity(p_full, pz, ptype)
                            p_time = calc_time(p_velocity, detector_distances[ind])
                            if pz > 0:
                                hits[ind][1].append([ptype, p_time, [pxn,pyn,pnorm]])
                            elif pz < 0:
                                hits[ind][0].append([ptype, p_time, [pxn,pyn,pnorm]])
                        

                        
    print("File #{} has been read".format(i))
  
events = sorted(events, key=lambda x: (x[0]))



# events[j]: j - number of event, events[j][2] - info, events[j][2][i] i - number of detector,
# events[j][2][i][0/1] - negative or positive Pz, events[j][2][i][0/1][k] - k - number of hit 
# events[j][2][i][0/1][k][0] - particle type, events[j][2][i][0/1][k][1] - time from event to hitting of detector
# events[j][2][i][0/1][k][2] - pxn, pyn, pnorm

File #1 has been read
File #2 has been read
File #3 has been read
File #4 has been read
File #5 has been read
File #6 has been read
File #7 has been read
File #8 has been read
File #9 has been read
File #10 has been read
File #11 has been read


In [7]:
print("Обработано {} событий".format(len(events)))

n_detectors = 2*n_rings
detector_width = detector_out_radius - detector_in_radius
number_of_hits = np.zeros(3)
# Среднее время прилета пионов для детекторов на [1, 1.7, 2,5] метрах, используется для нелинейного преобразования времени
avg_pion_times = [3.345, 5.675, 8.355]
max_hits = np.zeros(6, dtype=int)


one_ring_width = detector_width / n_radii
rings_width = []
# Разбиение детектора на n_radii одинаковых колец
for i in range(n_radii):
    rings_width.append(one_ring_width)

# Вычисление координат границ детекторов используя ширины колец
detector_borders = []
for i in range(len(rings_width)):
    rad = detector_in_radius
    for j in range(i,0,-1):
        rad += rings_width[j-1]
    detector_borders.append(rad)
detector_borders.append(detector_out_radius)
print("Границы радиального разбиения на {}".format(detector_borders))

event_data=np.zeros((n_detectors, n_angles, n_radii), dtype=np.float32)
if augmentation:
    augm_event_data=np.zeros((n_detectors, n_angles, n_radii), dtype=np.float32)
n = 0 
file_name = 'data/' + name_of_file + '.dat'
with open(file_name, 'w') as outf:
    outf.write('Maximum hits: ' + str(maximum_hits) + ' Lines: ' + str(n_lines) + '\n')
    for e in events:    # iterate over events
        event_data_2 = []
        if augmentation:
            augm_event_data_2 = []
        for i in range(n_lines): # листы: время, координата попадания, номер кольца
            event_data_2.append([])
            if augmentation:
                augm_event_data_2.append([])
        n += 1
        all_hits = e[2]    # all hits for one event
        b = e[0]
        randomizer = 2*np.random.random_sample() - 1

        det_num = 0
        for detector in all_hits:    # iterate over detectors
            dir_num = 0
            for direction in detector:    # iterate over positive and negative direction of axis Oz
                for hit in direction:    # iterate over hits in one direction
                    p_type = hit[0]

                    # Применяем нелинейное преобразование времени(записанного с заданной точностью), используя информацию о времени пролета пионов
                    particle_time = time_accuracy * ((hit[1] * 10**9)// time_accuracy)  
                    particle_value = 1 / (particle_time - avg_pion_times[det_num]) 

                    h = np.asarray(hit[2])
   
                    if h.shape[0] > 0:    
                        angle = np.arctan2(h[0], h[1])
                        radius = np.sqrt(h[0] ** 2 + h[1] ** 2)
                        angle_bin = np.floor(n_angles*(angle/2/np.pi+0.5))
                        angle_bin = angle_bin.astype(np.int)

                        # Вычисление радиальной координаты прилета частицы
                        rad = radius * detector_distances[det_num]
                        for i in range(len(detector_borders) - 1):
                            if detector_borders[i] < rad <= detector_borders[i+1]:
                                r_bin = i
                                break

                        if augmentation:
                            # augmented data
                            augm_angle = random_rotation(angle, np.pi, randomizer)
                            augm_angle_bin = np.floor(n_angles*(augm_angle/2/np.pi+0.5))
                            augm_angle_bin = augm_angle_bin.astype(np.int)
                            
                        if event_data[det_num * 2 + dir_num, angle_bin, r_bin] == 0:
                            event_data[det_num * 2 + dir_num, angle_bin, r_bin] = particle_value
                            event_data_2[0].append(particle_value)
                            event_data_2[1].append(r_bin * n_angles + angle_bin)
                            event_data_2[2].append(det_num * 2 + dir_num)
                            # event_data_2[3].append(get_particle_mass(p_type))
                            if augmentation:
                                augm_event_data_2[0].append(particle_value)
                                augm_event_data_2[1].append(r_bin * n_angles + augm_angle_bin)
                                augm_event_data_2[2].append(det_num * 2 + dir_num)
                                    
                            
                            number_of_hits[det_num] += 1
                        ''' # Использовалось для оценки количества повторных попаданий
                        else:
                            repeated_hits += 1
                            repeated_radii[det_num].append(radius*detector_distances[det_num])
                            repeated_distribution[det_num][r_bin] += 1
                        '''
                        
                dir_num += 1
            det_num += 1

        # Ниже идет запись данных в файл
        evdata2 = np.array(event_data_2)

        event_data_new = np.array(np.c_[evdata2, np.zeros((3, maximum_hits - evdata2.shape[1]))])
        outf.write(str(b) + ' ')
        output = np.reshape(event_data_new, (3, 1, -1)).astype(np.float32)      
        for j in range(output.shape[0]):    # write into file
            for i in range(output.shape[2]):
                outf.write(str(output[j][0][i]))
                outf.write(' ')
            outf.write('   ')
        outf.write('\n')

        if augmentation:
            augm_evdata2 = np.array(augm_event_data_2)
            augm_event_data_new = np.array(np.c_[augm_evdata2, np.zeros((3, maximum_hits - augm_evdata2.shape[1]))])
            outf.write(str(b) + ' ')
            augm_output = np.reshape(augm_event_data_new, (3, 1, -1)).astype(np.float32)      
            for j in range(augm_output.shape[0]):    # write into file
                for i in range(augm_output.shape[2]):
                    outf.write(str(augm_output[j][0][i]))
                    outf.write(' ')
                outf.write('   ')
            outf.write('\n')
        
        for j in range(output.shape[0]):    
            # print(j, np.count_nonzero(output[j]))
            max_hits[j] = max(np.count_nonzero(output[j]), max_hits[j])
        if n% int(len(events) / 10) == 0:
            print("Записано {:3.0f}% событий".format(n * 100 / len(events)))
            # Две строки в которых отслеживался прогресс записи и максимальное число попаданий
            '''print('{} from {}'.format(np.count_nonzero(output[0]), (len(e[2][0][0]) + len(e[2][0][1]) + len(e[2][1][0]) + len(e[2][1][1]) + len(e[2][2][0]) + len(e[2][2][1]) )))
            print("Максимальное количество попаданий по детекторам {}".format(max_hits))
            '''
        event_data*=0.0
        event_data = abs(event_data)
print("Максимальное количество попаданий по детекторам {}".format(max_hits))

Обработано 22000 событий
Границы радиального разбиения на [0.015, 0.0175, 0.020000000000000004, 0.022500000000000006, 0.025]
Записано  10% событий
Записано  20% событий
Записано  30% событий
Записано  40% событий
Записано  50% событий
Записано  60% событий
Записано  70% событий
Записано  80% событий
Записано  90% событий
Записано 100% событий
Максимальное количество попаданий по детекторам [53 50 42  0  0  0]
